# Unclaimed Property Records Lookup for California

```
Author: Amirhossein Kiani ( kiani [at] cs.stanford.edu )
Date Last Updated: Feb 12, 2020
Repository: https://github.com/akiani/linkedin-money
```


Thanks to the [escheat episode](https://www.npr.org/2020/01/24/799345159/episode-967-escheat-show) from Planet Money, I learned that there is $150+ of money awaiting me on the California Unclained Property website! 

So I quickly started searching my friends. But then I noticed I could search ALL my friends. Given that I am not Facebook anymore, LinkedIn is the only large social network that I currently have. So I wrote this script to get this done. 

You can do the same by running this Google Colab notebook and following these steps:

## Step 1: Download Your Connections

To do this:
1. Go to [here](https://www.linkedin.com/psettings/member-data)
2. Click on `Getting a copy of your data`
3. Under the `"Want something in particular?...` click on `Connections`
4. Click `Download...`
5. Wait for about 10 minutes and go back to the same page were the download will be available for you to get as a ZIP file.
6. Decompress the ZIP file on your laptop (it should result in a file called `Connections.csv`
7. In Google colab, click on the Folder icon on the left sidebar and then drag the CSV file there


## Step 2: Download Escheat Records
The most up-to-date unclaimed property records for California are freely available.

You can run the below cell to get these in Google Colab (this is around 3GB in size so it'll take about 30 min)

In [0]:
! wget https://dpupd.sco.ca.gov/01_From_0_To_Below_10.zip
! wget https://dpupd.sco.ca.gov/02_From_10_To_Below_100.zip
! wget https://dpupd.sco.ca.gov/03_From_100_To_Below_500.zip

These records are stored in ZIP files which each contain a large number of CSV files. Run the below cell to deflate the ZIP files so we can search them!

In [0]:
! unzip 01_From_0_To_Below_10.zip
! unzip 02_From_10_To_Below_100.zip
! unzip 03_From_100_To_Below_500.zip

Use a bit of data wrangling to get find the intersection of your LinkedIn friends and the records:

In [0]:
import pandas as pd
import glob

In [0]:
# read LinkedIn connections
connections = pd.read_csv("Connections.csv")

In [0]:
# create full names to be searched for
friends = []
for i, connection in connections.iterrows():
  first_name = connection["First Name"]
  last_name = connection["Last Name"]
  try:
    friends.append(first_name.upper() + " " + last_name.upper())
  except:
    # ignore NaN records
    pass


In [0]:
# go through each file in the records and print out potential matches

record_files = glob.glob("/content/*From*/*.csv")

results = pd.DataFrame()

for i, csv_file in enumerate(record_files):
  print("\nanalyzing file %i of %i\n" % (i+1, len(record_files)))
  try:
    claims = pd.read_csv(csv_file, encoding="cp1252", low_memory=False)
  except:
    # deal with different encodings in the records...
    claims = pd.read_csv(csv_file, encoding="cp850", low_memory=False)
  properties_found = claims[claims["OWNER_NAME"].isin(friends)]
  results = pd.concat([results, properties_found])
  if len(properties_found) == 0:
    print("no match found.")
  else:
    print("matches found:")
    for _, row in properties_found.iterrows():
      try:
        print("\t %s, $%.2f, %s, %s" %(
            row["OWNER_NAME"].title(),
            row["CASH_REPORTED"], 
            row["OWNER_CITY"].title(), 
            row["HOLDER_NAME"].title()))
      except:
        print("Error in rendering row", row)
        pass

# Analyzing the results

In [0]:
# print the total recovered cash $$$
results["CASH_REPORTED"].sum()

In [0]:
# only get the records with <5 matches to ignore super common names
less_than_5_match = pd.DataFrame()

for friend in friends:
  matches = results[results["OWNER_NAME"] == friend]
  if len(matches) > 0 and len(matches) < 5:
    less_than_5_match = pd.concat([less_than_5_match, matches])

In [0]:
less_than_5_match["CASH_REPORTED"].sum()

In [0]:
# print only the amounts records in <5 matches that are >$50
threshold = 50

total = 0

for i, row in less_than_5_match.iterrows():
  if row["CASH_REPORTED"] > threshold:
    total += row["CASH_REPORTED"]
    try:
      print("%s, $%.2f, %s, %s" %(
        row["OWNER_NAME"].title(),
        row["CASH_REPORTED"], 
        row["OWNER_CITY"].title(), 
        row["HOLDER_NAME"].title()))
    except:
      print("%s, $%.2f" %(
        row["OWNER_NAME"].title(),
          row["CASH_REPORTED"]))

In [0]:
total

Generating the visualizatons in the blog post:

In [0]:
# aggregate on holders over the whole results set
holders = pd.DataFrame()

for holder in results["HOLDER_NAME"].unique():
  results_for_holder = results[results["HOLDER_NAME"]== holder]
  holders = holders.append({"Holder": holder, "Sum": sum(
      results_for_holder["CASH_REPORTED"]), "Records": len(
          results_for_holder)}, ignore_index=True )

In [0]:
# aggregate on the owners over the whole results set
owners = pd.DataFrame()

for owner in results["OWNER_NAME"].unique():
  results_for_owner = results[results["OWNER_NAME"]== owner]
  owners = owners.append({"Owner": owner, "Sum": sum(
      results_for_owner["CASH_REPORTED"]), "Records": len(
          results_for_owner)}, ignore_index=True )

In [0]:
import plotly.express as px

In [0]:
fig = px.bar(holders.sort_values(
    "Sum", ascending=False).head(20), x="Holder", y="Sum", template="ggplot2")
fig.update_layout(xaxis_tickangle=-45)
fig.update_layout(
    title="Top 20 Holder Companies for Lost Properties",
    xaxis_title="Sum in Dollars",
    yaxis_title="Holder",
)

fig.show()

In [0]:
fig = px.histogram(owners, x="Sum", template="ggplot2")
fig.update_layout(xaxis_tickangle=-45)
fig.update_layout(
    title="Distribution Recovered Cash for LinkedIn Friends",
    xaxis_title="Dollar Amount",
    yaxis_title="Holder",
)

fig.show()